
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [3]:
# the path to your HDF5 formatted snps file
data = "./P_ni_6rm_v9.snps.hdf5"

In [4]:
imap = {
#"ref": ["reference"],
"Inam": ["P_ni_A7862_In", "P_ni_A7911_In", "P_ni_A7928_In", "P_ni_T6243_In",],
"Puru": ["P_ni_T5850_pu", "P_ni_T5940_pu", "P_ni_T5974_pu", "P_ni_T15938_pu","P_ni_80034_pu", "P_ni_T3609_pu", "P_ni_T3611_pu",  "P_ni_T3817_pu", "P_ni_T4043_pu", "P_ni_T4051_pu", "P_ni_T4313_pu", "P_ni_T4404_pu"],
"JiGu": ["P_ni_T22153_jigu"],
#"JiGu": [, "P_ni_T3261_jigu","P_ni_T15863_jigu", "P_ni_T15868_jigu", "P_ni_T15871_jigu", "P_ni_A3255_jigu",],
#"Mach": ["P_ni_T443_ma", "P_ni_T467_ma","P_ni_T369_ma","P_ni_J434_ma", "P_ni_J461_ma", "P_ni_J462_ma", "P_ni_J485_ma", "P_ni_J210_ma", "P_ni_J227_ma", "P_ni_J260_ma", "P_ni_A2418_ma",  "P_ni_A542_ma",],
#"Roar": ["P_ni_J684_roar", "P_ni_J724_roar","P_ni_J361_roar", "P_ni_J363_roar", "P_ni_J371_roar", "P_ni_J373_roar", "P_ni_J381_roar", "P_ni_J385_roar", "P_ni_J389_roar", "P_ni_J417_roar",],
#"ArSu": ["P_ni_J551_arsu", "P_ni_J602_arsu", "P_ni_J603_arsu", "P_ni_J614_arsu", "P_ni_J617_arsu","P_ni_80555_arsu","P_ni_86072_arsu", "P_ni_80684_arsu", "P_ni_80802_arsu", "P_ni_80874_arsu", "P_ni_85430_arsu",],
#"SuTa": ["P_ni_T14543_suta",  "P_ni_T9076_suta","P_ni_T16698_suta","P_ni_T10967_suta",  "P_ni_T11888_suta","P_ni_T10204_suta","P_ni_A15120_suta", "P_ni_77876_suta", "P_ni_78155_suta","P_ni_85721_suta",],
#"Para": ["P_ni_T1642_pa", "P_ni_T18703_pa","P_ni_T12345_pa", "P_ni_T12854_pa","P_ni_T11193_pa", "P_ni_T11222_pa","P_ni_T10673_pa", "P_ni_T10940_pa","P_ni_A7066_pa", "P_ni_A14342_pa", "P_ni_A15277_pa",]
}
      
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="P_ni_west_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 17
Sites before filtering: 1247688
Filtered (indels): 0
Filtered (bi-allel): 11467
Filtered (mincov): 1155347
Filtered (minmap): 1155347
Filtered (combined): 1156359
Sites after filtering: 91329
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 11:52:20 | running 70 structure jobs 


In [8]:
struct = ipa.structure(
    data=data, 
    name="P_ni_west_str", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [P_ni_west_str]


In [9]:
struct.get_clumpp_table(3)

[K3] 10/10 results permuted across replicates (max_var=0).


,0,1,2
P_ni_80034_pu,0.189,0.805,0.007
P_ni_A7862_In,0.176,0.799,0.026
P_ni_A7911_In,0.180,0.798,0.022
P_ni_A7928_In,0.181,0.794,0.025
P_ni_T15938_pu,0.198,0.790,0.012
P_ni_T22153_jigu,0.036,0.587,0.378
P_ni_T3609_pu,0.191,0.804,0.005
P_ni_T3611_pu,0.197,0.795,0.008
P_ni_T3817_pu,0.204,0.791,0.005
P_ni_T4043_pu,0.201,0.788,0.010


In [10]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-56513.52,623.216,0.00,0.00
3,10,43.638,-56511.10,814.027,2.42,35522.51
4,10,0.621,-92031.19,66784.334,-35520.09,41451.19
5,10,1.371,-86100.09,44574.653,5931.10,61105.32
6,10,0.844,-141274.31,87453.730,-55174.22,73803.61
7,10,1.051,-122644.92,49489.646,18629.39,52010.15
8,10,0.000,-156025.68,59059.545,-33380.76,0.00


In [11]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t940368841af4430a884d70dba2499fea" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 50000 100000 150000 estLnProbMean 0 20 40 60 deltaK

In [12]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [13]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [14]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5a09e4e49eff45c785eff75a98fcc63c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T6243_In P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T15938_pu P_ni_80034_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T22153_jigu 0.0 0.5 1.0

In [15]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [16]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [17]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tb6f5253a2ade4a09a24e767f75ac9ce5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T6243_In P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T15938_pu P_ni_80034_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T22153_jigu 0.0 0.5 1.0

In [18]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [19]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [20]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t7b4fa772f191478eb7ef7e4a80dffd84" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T6243_In P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T15938_pu P_ni_80034_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T22153_jigu 0.0 0.5 1.0

In [21]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [22]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [23]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2326e934ac4e46a3904eea5c9528c961" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T6243_In P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T15938_pu P_ni_80034_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T22153_jigu 0.0 0.5 1.0

In [24]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [25]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [26]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1e36a289bb714189b464af74889bd39a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T6243_In P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T15938_pu P_ni_80034_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T22153_jigu 0.0 0.5 1.0

# PCA

In [27]:
#!conda install scikit-learn -c conda-forge -y

In [28]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [29]:
imap = {
#"ref": ["reference"],
"Inam": ["P_ni_A7862_In", "P_ni_A7911_In", "P_ni_A7928_In", "P_ni_T6243_In",],
"Puru": ["P_ni_T5850_pu", "P_ni_T5940_pu", "P_ni_T5974_pu", "P_ni_80034_pu", "P_ni_T3609_pu", "P_ni_T3611_pu",  "P_ni_T3817_pu", "P_ni_T4043_pu", "P_ni_T4051_pu", "P_ni_T4313_pu", "P_ni_T4404_pu","P_ni_T15938_pu"],#
#"JiGu": ["P_ni_T22153_jigu", "P_ni_T3261_jigu","P_ni_T15863_jigu", "P_ni_T15868_jigu", "P_ni_T15871_jigu", "P_ni_A3255_jigu",],
#"Mach": ["P_ni_T443_ma", "P_ni_T467_ma","P_ni_T369_ma","P_ni_J434_ma", "P_ni_J461_ma", "P_ni_J462_ma", "P_ni_J485_ma", "P_ni_J210_ma", "P_ni_J227_ma", "P_ni_J260_ma", "P_ni_A2418_ma",  "P_ni_A542_ma",],
#"Roar": ["P_ni_J684_roar", "P_ni_J724_roar","P_ni_J361_roar", "P_ni_J363_roar", "P_ni_J371_roar", "P_ni_J373_roar", "P_ni_J381_roar", "P_ni_J385_roar", "P_ni_J389_roar", "P_ni_J417_roar",],
#"ArSu": ["P_ni_J551_arsu", "P_ni_J602_arsu", "P_ni_J603_arsu", "P_ni_J614_arsu", "P_ni_J617_arsu","P_ni_80555_arsu","P_ni_86072_arsu", "P_ni_80684_arsu", "P_ni_80802_arsu", "P_ni_80874_arsu", "P_ni_85430_arsu",],
#"SuTa": ["P_ni_T14543_suta",  "P_ni_T9076_suta","P_ni_T16698_suta","P_ni_T10967_suta",  "P_ni_T11888_suta","P_ni_T10204_suta","P_ni_A15120_suta", "P_ni_77876_suta", "P_ni_78155_suta","P_ni_85721_suta",],
#"Para": ["P_ni_T1642_pa", "P_ni_T18703_pa","P_ni_T12345_pa", "P_ni_T12854_pa","P_ni_T11193_pa", "P_ni_T11222_pa","P_ni_T10673_pa", "P_ni_T10940_pa","P_ni_A7066_pa", "P_ni_A14342_pa", "P_ni_A15277_pa",]
}
      
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [50]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 16
Sites before filtering: 1247688
Filtered (indels): 0
Filtered (bi-allel): 10854
Filtered (mincov): 765621
Filtered (minmap): 1153008
Filtered (combined): 1154004
Sites after filtering: 93684
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [51]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 15206/93684


In [52]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2ab333704910>,
 <toyplot.mark.Point at 0x2ab333897b10>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc8bd4bb280cb4c11aa006e2acc7daed3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_80034_pu P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T15938_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T6243_In -10 0 10 20 PC0 (9.3%) explained -10 0 10 20 30 PC1 (7.8%) explained Inam Puru

In [33]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("P_ni_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
P_ni_80034_pu,-8.48,-11.40,-9.97,11.83,-13.68,6.31,-7.83,11.03,-4.42,8.02
P_ni_A7862_In,17.93,1.02,-3.63,3.32,0.40,-3.87,1.90,-1.82,-12.27,5.42
P_ni_A7911_In,17.81,3.75,-5.48,-0.30,-1.74,1.55,-1.89,4.75,16.31,2.25
P_ni_A7928_In,20.89,3.52,1.45,0.62,1.18,3.48,-5.34,-1.68,-5.72,-3.90
P_ni_T15938_pu,-11.92,28.22,-11.80,1.69,-0.49,-1.01,-0.81,1.27,-1.00,0.27
P_ni_T3609_pu,-3.54,-5.45,-5.52,1.49,-10.92,-6.67,8.87,-10.63,13.83,-4.92
P_ni_T3611_pu,-5.84,-2.00,3.10,-3.31,10.23,-7.46,-9.78,0.31,0.41,-13.71
P_ni_T3817_pu,-1.74,3.58,14.58,-13.82,-9.32,5.34,6.02,-4.03,2.69,11.94
P_ni_T4043_pu,-3.13,-3.48,-3.26,-10.01,-11.41,-7.31,-1.73,-11.44,-13.47,-5.46
P_ni_T4051_pu,-3.33,-6.98,-7.26,-2.19,16.35,-10.77,4.71,-2.24,3.96,15.05


# Subsampling with replication

In [34]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 15206/93684


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t70ce99a54ef547e9a9e19314f36e0ccd" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_80034_pu P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T15938_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T6243_In -10 0 10 20 PC0 (9.1%) explained -10 0 10 20 30 PC1 (7.7%) explained Inam Puru

# Kmeans imputation (integer)

In [35]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Subsampling SNPs: 15206/93684


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tacaf6bbf495343fab080accd8130834f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_80034_pu P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T15938_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T6243_In -10 0 10 20 PC0 (9.2%) explained -20 -10 0 10 20 30 PC1 (7.6%) explained Inam Puru

In [36]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "P_ni_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [37]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [38]:
pca.run_tsne(subsample=True, perplexity=6.0, n_iter=100000, seed=945)

Subsampling SNPs: 15206/93684


In [39]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t617374fb3c9143149c0d6c5b12d9cf9d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> P_ni_80034_pu P_ni_A7862_In P_ni_A7911_In P_ni_A7928_In P_ni_T15938_pu P_ni_T3609_pu P_ni_T3611_pu P_ni_T3817_pu P_ni_T4043_pu P_ni_T4051_pu P_ni_T4313_pu P_ni_T4404_pu P_ni_T5850_pu P_ni_T5940_pu P_ni_T5974_pu P_ni_T6243_In -1000 -500 0 500 TNSE component 1 -200 0 200 400 600 TNSE component 2 Inam Puru

In [40]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "P_ni_TSNE_perp6.pdf")

# GENETIC DISTNANCES

In [41]:
import ipyrad.analysis as ipa
import toyplot

In [42]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 16
Sites before filtering: 1247688
Filtered (indels): 0
Filtered (bi-allel): 10854
Filtered (mincov): 765621
Filtered (minmap): 1153008
Filtered (combined): 1154004
Sites after filtering: 93684
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [43]:
# save to a CSV file
dist.dists.to_csv("P_ni_distances.csv")

In [44]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "P_ni_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [45]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [46]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t974be156f354486c99bd90cf36ea5898" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.134922 0.128080 0.146631 0.150335 0.148713 0.146546 0.153900 0.150239 0.148894 0.147784 0.149471 0.148585 0.148627 0.149182 0.158960 P_ni_A7862_In 1 0.134922 0.000000 0.130855 0.150260 0.153345 0.153239 0.148232 0.158661 0.152929 0.149748 0.148894 0.153207 0.150890 0.153004 0.152918 0.163101 P_ni_A7911_In 2 0.128080 0.130855 0.000000 0.148926 0.152395 0.151840 0.147944 0.158907 0.150890 0.149503 0.149588 0.149908 0.152289 0.151477 0.150026 0.162365 P_ni_A7928_In 3 0.146631 0.150260 0.148926 0.000000 0.143002 0.142212 0.142458 0.148937 0.145318 0.139939 0.138679 0.145596 0.142703 0.142233 0.143151 0.152566 P_ni_T6243_In 4 0.150335 0.153345 0.152395 0.143002 0.000000 0.144400 0.142490 0.148777 0.144923 0.142981 0.141380 0.144987 0.144443 0.143376 0.140515 0.155651 P_ni_T5850_pu 5 0.148713 0.153239 0.151840 0.142212 0.144400 0.000000 0.142468 0.146471 0.142020 0.141935 0.141977 0.144475 0.144187 0.142628 0.142735 0.154071 P_ni_T5940_pu 6 0.146546 0.148232 0.147944 0.142458 0.142490 0.142468 0.000000 0.146802 0.140365 0.140301 0.139469 0.141774 0.139330 0.142895 0.141892 0.153036 P_ni_T5974_pu 7 0.153900 0.158661 0.158907 0.148937 0.148777 0.146471 0.146802 0.000000 0.146802 0.144262 0.144859 0.147229 0.148862 0.147923 0.145809 0.157081 P_ni_80034_pu 8 0.150239 0.152929 0.150890 0.145318 0.144923 0.142020 0.140365 0.146802 0.000000 0.138679 0.139170 0.143525 0.141401 0.142447 0.142084 0.154253 P_ni_T3609_pu 9 0.148894 0.149748 0.149503 0.139939 0.142981 0.141935 0.140301 0.144262 0.138679 0.000000 0.141305 0.143461 0.139800 0.139138 0.142511 0.151178 P_ni_T3611_pu 10 0.147784 0.148894 0.149588 0.138679 0.141380 0.141977 0.139469 0.144859 0.139170 0.141305 0.000000 0.140921 0.140590 0.143130 0.137473 0.153505 P_ni_T3817_pu 11 0.149471 0.153207 0.149908 0.145596 0.144987 0.144475 0.141774 0.147229 0.143525 0.143461 0.140921 0.000000 0.144347 0.144539 0.142660 0.154978 P_ni_T4043_pu 12 0.148585 0.150890 0.152289 0.142703 0.144443 0.144187 0.139330 0.148862 0.141401 0.139800 0.140590 0.144347 0.000000 0.143632 0.141817 0.154028 P_ni_T4051_pu 13 0.148627 0.153004 0.151477 0.142233 0.143376 0.142628 0.142895 0.147923 0.142447 0.139138 0.143130 0.144539 0.143632 0.000000 0.141839 0.156505 P_ni_T4313_pu 14 0.149182 0.152918 0.150026 0.143151 0.140515 0.142735 0.141892 0.145809 0.142084 0.142511 0.137473 0.142660 0.141817 0.141839 0.000000 0.152961 P_ni_T4404_pu 15 0.158960 0.163101 0.162365 0.152566 0.155651 0.154071 0.153036 0.157081 0.154253 0.151178 0.153505 0.154978 0.154028 0.156505 0.152961 0.000000 P_ni_T15938_pu